# Week 5 Statistical Inference with Python

In week6, we've covered:
* **Variance**:  
    * Measuring Uncertainty  
    * Prior and Posterior Distributions  
    * Credible Intervals  
* **Inference**:  
    * Hypothesis Testing  
    * False Discovery Rate  

The best way to consolidate the knowledge in your mind is by practicing.<br>Please complete the part marked with <span style="color:green">**# TODO**</span>.

[Google](www.google.com) and [Python Documentation](https://docs.python.org/3/contents.html) are your good friends if you have any python questions.

Upload **Week6_Statistical_Inference_With_Python_Homework.ipynb** notebook to your Google Drive and open it with Google Colab

## Inference  

Recall that last week, we were introduced to statistical inference using batting averages from baseball as a running example. We used the binomial distribution to model the likelihood that a player would get a hit during an at bat, given historical data. We used the beta distribution to estimate batting averages and measure our uncertainty.  

This covered the first two articles of this series titled "The beta distribution" and "Emperical Bayes estimation": http://varianceexplained.org/r/simulation-bayes-baseball/  

Let's continue the discussion of variance, and discuss how to use variance to come up with a range for our estimates of batting averages. By the end of this exercise, you should feel comfortable giving a probability that an estimate lies in an interval. Read about credible intervals in the third article of the series: http://varianceexplained.org/r/credible_intervals_baseball/  

### Exercises  

1. Run the codeblock below to reproduce the dataset from last week.  Print out `alpha0` and `beta0`.

In [ ]:
import pandas as pd

# read the dataset
batting_url = 'https://raw.githubusercontent.com/chadwickbureau/baseballdatabank/master/core/Batting.csv'
pitching_url = 'https://raw.githubusercontent.com/chadwickbureau/baseballdatabank/master/core/Pitching.csv'
people_url = 'https://raw.githubusercontent.com/chadwickbureau/baseballdatabank/master/core/People.csv'

batting = pd.read_csv(batting_url)
pitching = pd.read_csv(pitching_url)
master = pd.read_csv(people_url)

# recreate career dataframe
pitchers = pitching['playerID'].tolist()
batting = batting[batting['AB'] > 0]
batting = batting[~batting['playerID'].isin(pitchers)] #filtered out pitchers
batting_sum =batting.groupby(['playerID']).agg({'H':'sum','AB':'sum'}) # get total hits and at bats for each player
batting_sum['Avg'] = batting_sum.loc[:,'H'] / batting_sum.loc[:,'AB'] #calculate the avg batting rate
career = pd.merge(batting_sum, master, how='inner', on='playerID')[["playerID","nameFirst", "nameLast", "H", "AB", "Avg"]]
career= career[career["AB"]>=500]
career.head()

# helper function to estimate priors for alpha and beta
def moments(mu, sigma2):
    alpha = mu**2 * ((1 - mu) / sigma2 - 1 / mu)
    beta = alpha * (1 / mu - 1)
    return (alpha, beta)

# get priors for alpha and beta
empirical_mean = career['Avg'].to_numpy().mean()
empirical_variance = career['Avg'].to_numpy().var()
alpha0, beta0 = moments(empirical_mean, empirical_variance)

# use priors to update estimate of batting average for each player
career['eb'] = ((career.loc[:,'H'] + alpha0) / (career.loc[:,'AB'] + alpha0 + beta0))

# concatenate first and last name
career['name'] = career.loc[:, 'nameFirst'] + ' ' + career.loc[:, 'nameLast']
career.sort_values('eb', ascending=False).loc[:,["playerID", 'name', 'H', 'AB', 'Avg', 'eb']].head(5)

,playerID,name,H,AB,Avg,eb
4106,hornsro01,Rogers Hornsby,2930,8173,0.358497,0.354854
4296,jacksjo01,Shoeless Joe Jackson,1772,4981,0.355752,0.350070
2202,delahed01,Ed Delahanty,2597,7510,0.345806,0.342354
3619,hamilbi01,Billy Hamilton,2164,6283,0.344421,0.340392
3818,heilmha01,Harry Heilmann,2660,7787,0.341595,0.338422


In [ ]:
print(alpha0,beta0)

79.7461639859911 228.90413176803585


2. Recall `alpha0` and `beta0` represent prior belief for the "average player" as determined by the entire data set. Use your knowledge from last week to generate an array of `1000` samples from a beta distribution with parameters `alpha0` and `beta0`.  

  2a. Calculate the sample mean and compare it to the ratio $\frac{\alpha_0}{\alpha_0 + \beta_0}$

In [ ]:
from scipy.stats import beta
import numpy as np
from itertools import repeat

prior_samples = beta.rvs(a=alpha0,b=beta0,size=1000)
prior_mean = np.mean(prior_samples)
expected_value_prior = alpha0/(alpha0+beta0)
print("Prior Sample Mean: " + str(prior_mean))
print("Expected Value Prior: " + str(expected_value_prior))
print("Difference: " + str(prior_mean-expected_value_prior))
# Close but not the same

Prior Sample Mean: 0.25893564311237377
Expected Value Prior: 0.25837060609701573
Difference: 0.0005650370153580386


  2b. Calculate the 0.10 and 0.90 quantlies of the sample.  

In [ ]:
def print_quantile(quantile, samples):
    q = np.quantile(a=samples, q=quantile)
    s = "The " + str(quantile * 100) + "% quantile of the sample is: " + str(q)
    print(s)
    return q

list(map(print_quantile,[0.1, 0.9], repeat(prior_samples, 2)))

The 10.0% quantile of the sample is: 0.2270291612539028
The 90.0% quantile of the sample is: 0.29146352047955026


[0.2270291612539028, 0.29146352047955026]

3. Find two players in the `career` dataframe that have a similar `Avg` but very different at bats (`AB`).

  3a. Create a two-row dataframe that includes only these two players, and display their `name`, `H`, `AB`, `Avg`, and estimated avg (`eb`).

In [ ]:
## Find upper and lower AB ranges
upper_ab_range = np.quantile(career['AB'], 0.75)
lower_ab_range = np.quantile(career['AB'], 0.25)

## Looks for players with a similar AB
## Find one player from that subset in the upper AB range and another in the lower AB range
near_285_avg = career.query('Avg > 0.305 & Avg < 0.310')
upper_ab_player_id = np.random.choice(
    near_285_avg.query('AB > @upper_ab_range')['playerID']
    )
lower_ab_player_id = np.random.choice(
    near_285_avg.query('AB < @lower_ab_range')['playerID']
    )

player_stats = near_285_avg.query('playerID in [@upper_ab_player_id, @lower_ab_player_id]')[['name', 'H', 'AB', 'Avg', 'eb']].reset_index(drop=True)
player_stats

,name,H,AB,Avg,eb
0,Ben Paschal,243,787,0.308767,0.294570
1,John Stone,1391,4494,0.309524,0.306236


3b. Calculate the *posterior* values for $\alpha$ and $\beta$ for the two players in your subset.

(This Stack Overflow answer details the update process for $\alpha$ and $\beta$: https://stats.stackexchange.com/a/47782)

In [ ]:
player_stats_post_params = player_stats.assign(alpha = alpha0 + player_stats['H'],
                                               beta = beta0 + player_stats['AB'] - player_stats['H'])
player_stats_post_params

,name,H,AB,Avg,eb,alpha,beta
0,Ben Paschal,243,787,0.308767,0.294570,322.746164,772.904132
1,John Stone,1391,4494,0.309524,0.306236,1470.746164,3331.904132


  3c. Though the `Avg` and the priors (`alpha0`, `beta0`) are similar for your two players, their estimated variance should be very different (due to the difference in at bats). Make an argument (no proof required): Which of your two players will have a lower 0.10 quantile for the posterior distribution of their estimated batting average (`eb`)? Explain your reasoning in a sentence or two.

In [ ]:
worse_player_name = player_stats[player_stats.AB == player_stats.AB.min()]['name']
print("The worse player, " + worse_player_name + ", will have the lowest 0.10 quantile for the posterior ditribution of their estimated batting average due to the higher variance of the posterior distribution.")



0    The worse player, Ben Paschal, will have the l...
Name: name, dtype: object


3d. Use your posterior alphas and betas to generate a sample of `1000` estimated batting averages for each of your two players. Calculate the 0.10 and 0.90 quantile for each sample distribution. Compare the outcome to your prediction in 3c. Reconcile any differences.

In [ ]:
def beta_quantile(row, quant, n=int(5e4)):
    post_dist = beta.rvs(row['alpha'], row['beta'], size=n)
    return np.quantile(post_dist, quant)

player_stats_post_params['eb_10_quantile'] = player_stats_post_params.apply(lambda x: beta_quantile(x, 0.1), axis=1)
player_stats_post_params['eb_90_quantile'] = player_stats_post_params.apply(lambda x: beta_quantile(x, 0.9), axis=1)
player_stats_post_params

,name,H,AB,Avg,eb,alpha,beta,eb_10_quantile,eb_90_quantile
0,Ben Paschal,243,787,0.308767,0.294570,322.746164,772.904132,0.276977,0.312287
1,John Stone,1391,4494,0.309524,0.306236,1470.746164,3331.904132,0.297723,0.314793


4. Finally, let's compare confidence and credible intervals.

  4a. For each of your two players, use their `H` and `AB` to calculate the 95% binomial proportion confidence interval.
  
  (The wikipedia entry will be helpful: https://en.wikipedia.org/wiki/Binomial_proportion_confidence_interval. Note that for a 95% confidence interval, the *z* score is 1.96.)

In [ ]:
from arviz.stats import hdi
import math

def confidence_interval(row):
    p_hat = row['H']/row['AB']
    z_score = 1.96
    sd = z_score * math.sqrt((p_hat * (1 - p_hat))/row['AB'])
    return [p_hat - sd, p_hat + sd]

player_stats_post_params['confidence_inter_96%'] = player_stats_post_params.apply(confidence_interval, axis=1)

4b. Calculate the 95% credible interval using `alpha` and `beta`. Compare the difference between the confidence interval and the credible interval. When are they the most different?

(Here is another resource for understanding confidence vs credible intervals: https://towardsdatascience.com/do-you-know-credible-interval-e5b833adf399)

In [ ]:
def hdci_95(row, n=int(5e4)):
    post_dist = beta.rvs(row['alpha'], row['beta'], size=n)
    return hdi(ary=post_dist, hdi_prob=0.95)

player_stats_post_params['credible_interval_95%'] = player_stats_post_params.apply(hdci_95, axis=1)

player_stats_post_params
# The prior has a much stronger effect on the worse player since there is less observational data. Here, the largest difference is observed when there is the least data (at-bats.)

,name,H,AB,Avg,eb,alpha,beta,eb_10_quantile,eb_90_quantile,confidence_inter_96%,credible_interval_95%
0,Ben Paschal,243,787,0.308767,0.294570,322.746164,772.904132,0.276977,0.312287,"[0.276490218925284, 0.3410447238955546]","[0.26720547901085073, 0.3210351822668962]"
1,John Stone,1391,4494,0.309524,0.306236,1470.746164,3331.904132,0.297723,0.314793,"[0.29600741306072886, 0.3230402059868902]","[0.2930389356133834, 0.3190492649006187]"


In [ ]:
#alternative implementation for credible interval
import numpy as np

post_dist = np.random.beta(301.803363,733.847297, 50000)
np.quantile(post_dist, [0.05,0.95]) # assumes centered distribution i think

array([0.26849917, 0.31487856])

## Hypothesis Testing

Now that you have a handle on credible intervals, you are ready to use those intervals to test hypotheses.

Read the fourth article to begin to understand how credible intervals can be useful to make decisions: http://varianceexplained.org/r/bayesian_fdr_baseball/

### Exercises  

5. Let's aggregate Hank Aaron's seasons to get his total career at bats.

In [ ]:
h_aaron = career[career['playerID'] == 'aaronha01']
h_aaron = h_aaron.assign(alpha = alpha0+h_aaron["H"], beta = beta0 +h_aaron["AB"] - h_aaron["H"])
h_aaron.head()

,playerID,nameFirst,nameLast,H,AB,Avg,eb,name,alpha,beta
0,aaronha01,Hank,Aaron,3771,12364,0.304998,0.303863,Hank Aaron,3850.746164,8821.904132


5a. Calculate Hank Aaron's total Hits (`H`) and At Bats (`AB`)

In [ ]:
h_aaron[['name', 'H', 'AB', 'alpha', 'beta']]

,name,H,AB,alpha,beta
0,Hank Aaron,3771,12364,3850.746164,8821.904132


5b. Use random sampling to estimate the probability that Hank Aaron's career batting average is below 0.300.

In [ ]:
def less_than_300(alpha, beta_val):
    posterior = beta.rvs(a=alpha,b=beta_val,size=int(5e4))
    return np.mean(posterior < 0.3)

h_aaron_perc = less_than_300(h_aaron['alpha'], h_aaron['beta'])
str(h_aaron_perc * 100) + "%` chance Hank Arrons batting average is below 0.300."

'17.454%` chance Hank Arrons batting average is below 0.300.'

5c. Use random sampling to estimate the probability that the *average* player has a career batting average below 0.300

In [ ]:
avg_perc = less_than_300(alpha0, beta0)
str(avg_perc * 100) + "%` chance the average player's batting average is below 0.300."

"94.99%` chance the average player's batting average is below 0.300."

5d. How would you use the estimates from 4b and 4c to describe how good a hitter Hank Aaron was?

In [ ]:
# You could calculate the % difference between the samples from the prior and the posterior to get an estimate of how much better he was than the average player with a certain level of confidence.

6. Find two rows of the `career` dataframe that have the same `eb` estimate (to the third decimal place). Compare the posterior error probability that each of these players has a batting average below 0.300. Reconcile any differences.

In [ ]:
def less_than_300_rowwise(row):
    return less_than_300(alpha=row['alpha'], beta_val=row['beta'])

two_players = career \
    .assign(eb_rounded = round(career['eb'],3)) \
    .query('eb_rounded == 0.225') \
    .sample(2)

two_players_post_params = two_players \
    .assign(alpha = alpha0 + two_players['H'],
            beta = beta0 + two_players['AB'] - two_players['H'])

two_players_post_params['prob_less_than_300'] = two_players_post_params.apply(less_than_300_rowwise, axis=1)
two_players_post_params[['name', 'H', 'AB', 'alpha', 'beta', 'eb', 'prob_less_than_300']].reset_index(drop=True)

,name,H,AB,alpha,beta,eb,prob_less_than_300
0,Rafael Belliard,508,2301,587.746164,2021.904132,0.225220,1.0
1,Charlie O'Brien,493,2232,572.746164,1967.904132,0.225433,1.0


## Submission

Download completed **Week_5_Statistical_Inference_With_Python_Homework.ipynb** from Google Colab and commit to your personal Github repo you shared with the faculty.